In [1]:
!pip install stable-baselines3[extra]
!pip install --user --upgrade git+http://github.com/pyglet/pyglet@pyglet-1.5-maintenance

# !pip install gym
# !pip install pyglet


[notice] A new release of pip available: 22.2.2 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: Can not perform a '--user' install. User site-packages are not visible in this virtualenv.

[notice] A new release of pip available: 22.2.2 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import os 
import gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy 

ModuleNotFoundError: No module named 'gym'

In [3]:
environment_name = "CartPole-v0"
env = gym.make(environment_name)

In [4]:
episodes = 1
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0

    while not done:
        env.render(mode='human')
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score += reward
    print(f"Episode: {episode} Score: {score}")
env.close()

Episode: 1 Score: 52.0


In [5]:
env.action_space.sample()

1

In [6]:
env.observation_space.sample()

array([-3.0886024e-01,  2.8985995e+38, -3.7125295e-01,  4.3916335e+37],
      dtype=float32)

## Training an RL Model

In [7]:
training_folder = "training"
os.makedirs(training_folder, exist_ok=True)
log_path = os.path.join(training_folder, "logs")
saved_models_path = os.path.join(training_folder, "saved_models")

os.makedirs(log_path, exist_ok=True)
os.makedirs(saved_models_path, exist_ok=True)


In [8]:
env = gym.make(environment_name)
env = DummyVecEnv([lambda: env])
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

Using cpu device


In [2]:
x = [1,2,4,5]
y = [i for i, num in enumerate(x)]
print(y)

[0, 1, 2, 3]


In [9]:
model.learn(total_timesteps=20_000)

Logging to training\logs\PPO_1
-----------------------------
| time/              |      |
|    fps             | 1790 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 1263         |
|    iterations           | 2            |
|    time_elapsed         | 3            |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0083049685 |
|    clip_fraction        | 0.0913       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.686       |
|    explained_variance   | 0.000131     |
|    learning_rate        | 0.0003       |
|    loss                 | 7.22         |
|    n_updates            | 10           |
|    policy_gradient_loss | -0.0142      |
|    value_loss           | 57.8         |
----------------------------

Save and reload model

In [10]:
PPO_Path = os.path.join("training", 'saved_models', 'PPO_Model_Cartpole')

In [11]:
model.save(PPO_Path)

In [12]:
del model 

In [13]:
model = PPO.load(PPO_Path, env=env)

## Evalutation

In [14]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

C:\Users\MikePratt\AppData\Local\Programs\Python\Python310\lib\site-packages\stable_baselines3\common\evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


(200.0, 0.0)

In [15]:
env.close()

## Test

In [16]:
episodes = 2
for episode in range(1, episodes+1):
    obs = env.reset()
    done = False
    score = 0

    while not done:
        env.render(mode='human')
        action, _ = model.predict(obs)
        obs, reward, done, info = env.step(action)
        score += reward
    print(f"Episode: {episode} Score: {score}")
env.close()

Episode: 1 Score: [200.]
Episode: 2 Score: [200.]


In [17]:
env.close()

## Viewing logs in tensorboard

In [20]:
training_log_path = os.path.join(log_path, 'PPO_1')

In [21]:
training_log_path

'training\\logs\\PPO_1'

In [24]:
!tensorboard --logdir={training_log_path} --port=5000



^C


## Adding a callback to the training stage

In [25]:
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold

In [27]:
save_path = os.path.join('training', 'saved_models')

In [28]:
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=200, verbose=1)
eval_callback = EvalCallback(env,
                            callback_on_new_best=stop_callback,
                            eval_freq=10000,
                            best_model_save_path=save_path,
                            verbose=1)

In [ ]:
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

In [29]:
model.learn(total_timesteps=20000, callback=eval_callback)

Logging to training\logs\PPO_2
-----------------------------
| time/              |      |
|    fps             | 1905 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 1233         |
|    iterations           | 2            |
|    time_elapsed         | 3            |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0028930644 |
|    clip_fraction        | 0.00654      |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.539       |
|    explained_variance   | 0.112        |
|    learning_rate        | 0.0003       |
|    loss                 | 16.1         |
|    n_updates            | 110          |
|    policy_gradient_loss | -0.00211     |
|    value_loss           | 41           |
----------------------------

## Changing policies

In [34]:
net_arch = dict(pi=[128,128,128,128], vf=[128,128,128,128])

In [35]:
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path, policy_kwargs={'net_arch':net_arch})

Using cpu device


In [36]:
model.learn(total_timesteps=20000, callback=eval_callback)

Logging to training\logs\PPO_3
-----------------------------
| time/              |      |
|    fps             | 1369 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
----------------------------------------
| time/                   |            |
|    fps                  | 802        |
|    iterations           | 2          |
|    time_elapsed         | 5          |
|    total_timesteps      | 4096       |
| train/                  |            |
|    approx_kl            | 0.01521586 |
|    clip_fraction        | 0.208      |
|    clip_range           | 0.2        |
|    entropy_loss         | -0.682     |
|    explained_variance   | 0.00234    |
|    learning_rate        | 0.0003     |
|    loss                 | 4.48       |
|    n_updates            | 10         |
|    policy_gradient_loss | -0.0245    |
|    value_loss           | 19.9       |
----------------------------------------
---------------------

## Using an alternate algorithm

In [37]:
from stable_baselines3 import DQN

In [38]:
model = DQN('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

Using cpu device


In [39]:
model.learn(total_timesteps=20000)

Logging to training\logs\DQN_1
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.962    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 7005     |
|    time_elapsed     | 0        |
|    total_timesteps  | 80       |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.925    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 7047     |
|    time_elapsed     | 0        |
|    total_timesteps  | 158      |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.867    |
| time/               |          |
|    episodes         | 12       |
|    fps              | 7878     |
|    time_elapsed     | 0        |
|    total_timesteps  | 279      |
----------------------------------
------------------------

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 108      |
|    fps              | 10086    |
|    time_elapsed     | 0        |
|    total_timesteps  | 2406     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 112      |
|    fps              | 10099    |
|    time_elapsed     | 0        |
|    total_timesteps  | 2500     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 116      |
|    fps              | 10140    |
|    time_elapsed     | 0        |
|    total_timesteps  | 2639     |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 216      |
|    fps              | 10074    |
|    time_elapsed     | 0        |
|    total_timesteps  | 4701     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 220      |
|    fps              | 10092    |
|    time_elapsed     | 0        |
|    total_timesteps  | 4780     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 224      |
|    fps              | 10119    |
|    time_elapsed     | 0        |
|    total_timesteps  | 4904     |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 324      |
|    fps              | 10308    |
|    time_elapsed     | 0        |
|    total_timesteps  | 7042     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 328      |
|    fps              | 10332    |
|    time_elapsed     | 0        |
|    total_timesteps  | 7162     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 332      |
|    fps              | 10343    |
|    time_elapsed     | 0        |
|    total_timesteps  | 7242     |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 432      |
|    fps              | 10444    |
|    time_elapsed     | 0        |
|    total_timesteps  | 9251     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 436      |
|    fps              | 10446    |
|    time_elapsed     | 0        |
|    total_timesteps  | 9326     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 440      |
|    fps              | 10443    |
|    time_elapsed     | 0        |
|    total_timesteps  | 9386     |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 540      |
|    fps              | 10521    |
|    time_elapsed     | 1        |
|    total_timesteps  | 11472    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 544      |
|    fps              | 10482    |
|    time_elapsed     | 1        |
|    total_timesteps  | 11544    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 548      |
|    fps              | 10496    |
|    time_elapsed     | 1        |
|    total_timesteps  | 11654    |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 648      |
|    fps              | 10534    |
|    time_elapsed     | 1        |
|    total_timesteps  | 13760    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 652      |
|    fps              | 10516    |
|    time_elapsed     | 1        |
|    total_timesteps  | 13830    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 656      |
|    fps              | 10511    |
|    time_elapsed     | 1        |
|    total_timesteps  | 13897    |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 756      |
|    fps              | 10621    |
|    time_elapsed     | 1        |
|    total_timesteps  | 16125    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 760      |
|    fps              | 10625    |
|    time_elapsed     | 1        |
|    total_timesteps  | 16216    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 764      |
|    fps              | 10613    |
|    time_elapsed     | 1        |
|    total_timesteps  | 16343    |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 864      |
|    fps              | 10532    |
|    time_elapsed     | 1        |
|    total_timesteps  | 18646    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 868      |
|    fps              | 10539    |
|    time_elapsed     | 1        |
|    total_timesteps  | 18744    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 872      |
|    fps              | 10542    |
|    time_elapsed     | 1        |
|    total_timesteps  | 18844    |
----------------------------------
----------------------------------
| rollout/          